In [8]:
import pandas as pd
import numpy as np
ufos = pd.read_csv('/Users/Javi/Desktop/lab-model-deployment/complete.csv', on_bad_lines='skip')

ufos.head()

/var/folders/0b/n26mz9yj7kq1tm2fxmm799bc0000gp/T/ipykernel_5049/900378384.py:3: DtypeWarning: Columns (5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  ufos = pd.read_csv('/Users/Javi/Desktop/lab-model-deployment/complete.csv', on_bad_lines='skip')


,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667
3,10/10/1956 21:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833
4,10/10/1960 20:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611


In [9]:
ufos = pd.DataFrame({'Seconds': ufos['duration (seconds)'], 'Country': ufos['country'],'Latitude': ufos['latitude'],'Longitude': ufos['longitude']})

ufos.Country.unique()

array(['us', nan, 'gb', 'ca', 'au', 'de'], dtype=object)

In [11]:
ufos['Seconds'] = pd.to_numeric(ufos['Seconds'], errors='coerce')

# Eliminar filas con NaN en 'Seconds'
ufos.dropna(subset=['Seconds'], inplace=True)

# Filtrar valores entre 1 y 60
ufos = ufos[(ufos['Seconds'] >= 1) & (ufos['Seconds'] <= 60)]

# Verificar el DataFrame
ufos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25861 entries, 2 to 88675
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Seconds    25861 non-null  float64
 1   Country    25861 non-null  object 
 2   Latitude   25861 non-null  object 
 3   Longitude  25861 non-null  float64
dtypes: float64(2), object(2)
memory usage: 1010.2+ KB


In [12]:
from sklearn.preprocessing import LabelEncoder

ufos['Country'] = LabelEncoder().fit_transform(ufos['Country'])

ufos.head()

,Seconds,Country,Latitude,Longitude
2,20.0,3,53.2,-2.916667
3,20.0,4,28.9783333,-96.645833
14,30.0,4,35.8238889,-80.253611
24,60.0,4,45.5827778,-122.352222
25,3.0,3,51.783333,-0.783333


In [13]:
from sklearn.model_selection import train_test_split

Selected_features = ['Seconds','Latitude','Longitude']

X = ufos[Selected_features]
y = ufos['Country']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [14]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)
predictions = model.predict(X_test)

print(classification_report(y_test, predictions))
print('Predicted labels: ', predictions)
print('Accuracy: ', accuracy_score(y_test, predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        40
           1       0.85      0.39      0.53       251
           2       1.00      1.00      1.00         8
           3       1.00      1.00      1.00       146
           4       0.97      1.00      0.98      4728

    accuracy                           0.97      5173
   macro avg       0.96      0.88      0.90      5173
weighted avg       0.96      0.97      0.96      5173

Predicted labels:  [4 4 4 ... 4 4 4]
Accuracy:  0.9669437463754108


/Users/Javi/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [15]:
import pickle
model_filename = 'ufo-model.pkl'
pickle.dump(model, open(model_filename,'wb'))

model = pickle.load(open('ufo-model.pkl','rb'))
print(model.predict([[50,44,-12]]))

[3]


/Users/Javi/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
